In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import numpy
import pandas
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

#from keras.models import Sequential
from tensorflow.keras.layers import Dense ,Dropout,BatchNormalization
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

ModuleNotFoundError: No module named 'tensorflow'

In [5]:
# df=pd.read_csv("/data/Admission_data.csv")
# #changing names because previous names are little bit confusing
# df=df.rename(index=str, columns={"GRE Score": "GRE", "TOEFL Score": "TOEFL", "Chance of Admit ": ""})
# #we donot need serial number so its good to drop it because its just a number
# df=df.drop("Serial No.",axis=1)
# df.head(10)

In [ ]:
from refractio import snowflake
df = snowflake.get_dataframe("ADMISSION500MB")
df=df.rename(index=str, columns={"GRE_SCORE": "GRE", "TOEFL_SCORE": "TOEFL", "CHANCE_OF_ADMIT ": "Admission_Chance"})
#we donot need serial number so its good to drop it because its just a number
df=df.drop("SERIAL_NO",axis=1)
df.head(10)

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


In [7]:
admit=np.asarray(df["CHANCE_OF_ADMIT"])
len(np.unique(admit))
#we have 60 different values in the coloum [chance to predict]

81

In [ ]:
X=np.asarray(df.drop("CHANCE_OF_ADMIT",axis=1))
Y=np.asarray(df["CHANCE_OF_ADMIT"])

X_train, X_test, y_train, y_test = train_test_split(
     X,Y, test_size=0.2, random_state=0)
from sklearn.preprocessing import MinMaxScaler
scaler =  MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [ ]:
model_keras = tf.keras.models.Sequential(
    [
        tf.keras.layers.Dense(16, input_dim=7, activation='relu'),
        tf.keras.layers.Dense(8, input_dim=7, activation='relu'),
        tf.keras.layers.Dense(1)  
    ]
)
model_keras.summary()

In [ ]:
# Compile model
model_keras.compile(
    optimizer='adam',
    #loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    loss=tf.keras.losses.MeanAbsolutePercentageError(),
    metrics=["accuracy"]
)

In [ ]:
 # Train model
model_keras.fit(X_train, y_train, batch_size=8, epochs=1, validation_split=0.2,
          callbacks=[EarlyStopping(monitor='val_loss', patience=4)])      

In [ ]:
# Test model
test_scores = model_keras.evaluate(X_test, y_test, verbose=2)
print("Test loss: ", test_scores[0])
print("Test accuracy: ", test_scores[1])

In [ ]:
# define base model
'''
def baseline_model():
    # create model
    model = Sequential()
    
    
    
    model.add(Dense(16, input_dim=7, activation='relu'))
    
    
    model.add(Dense(8, input_dim=7, activation='relu'))
    
    
    model.add(Dense(1))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
'''

In [ ]:
#estimator = KerasRegressor(build_fn=baseline_model, epochs=30, batch_size=3, verbose=1)

In [ ]:
#estimator.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import accuracy_score
#prediction = model_kr.predict(X_test)
#prediction = model_keras.predict(X_test)

In [ ]:
# train_error =  np.abs(y_test - prediction)
# mean_error = np.mean(train_error)
# min_error = np.min(train_error)
# # max_error = np.max(train_error)
# std_error = np.std(train_error)

In [ ]:
# print("std_error: ",std_error)
# print("mean_error: ",mean_error)
# print("min_error: ",min_error)
# print("max_error: ",max_error)

In [ ]:
#Model function
def model(algorithm,dtrain_X,dtrain_Y,dtest_X,dtest_Y,cols=None):

    #algorithm.fit(dtrain_X,dtrain_Y)
    predictions = algorithm.predict(dtest_X)
    prediction_probabilities = algorithm.predict(dtest_X)
    print (algorithm)
    
    return predictions,prediction_probabilities,algorithm

In [ ]:
#y_pred,y_prob,model_obj=model(estimator,X_train,y_train,X_test,y_test)
y_pred,y_prob,model_obj=model(model_keras,X_train,y_train,X_test,y_test)

In [ ]:
from refractml import *
from refractml.constants import MLModelFlavours

In [ ]:
@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data = pd.DataFrame(eval(payload))
    prediction = model.predict(data)
    #prediction = pd.Series(model.predict(data))
    print (prediction)
    #return prediction
    return prediction.tolist()

In [ ]:
cols = ['GRE Score','TOEFL Score','University Rating','SOP','LOR','CGPA','Research']

In [ ]:
X_temp = pd.DataFrame(X,columns=cols)

In [ ]:
import requests
req = requests.Request()
req.json = {"payload":X_temp.head(1).to_json(orient = 'records')}

In [ ]:
req.json['payload']

In [ ]:
req.json

In [ ]:
#sch = generate_schema(score,(model_obj, req),X_temp)
sch = generate_schema(score,(model_keras, req),X_temp)
print(sch)

In [ ]:
register_model(model_keras, score, name="Admission_RegressionKERAS", 
                description="Identify chances of Admission",
                flavour=MLModelFlavours.keras)
'''
               schema=sch,
                y_true=y_test, y_pred = y_pred, #y_pred=pd.Series(y_pred), 
                prob=y_pred, features=cols, 
               labels=["Admission_Chance"], 
               init_script="" ,
                model_type="regression", 
                input_type="json", explain_ai=True, x_train=X_train, x_test=X_test, 
                y_train=y_train, y_test=y_test,
                feature_names=cols,feature_ids=cols,
                target_names=["Admission_Chance"], 
                kyd=False, kyd_score =False)
'''
print("Registering model")